In [0]:
!pip install mnist

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp "/content/drive/My Drive/imimic/wann/mnist_test_c_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_cn_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_features.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_hu_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_test_radiomics.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_c_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_cn_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_features.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_hu_moments.npy" .
!cp "/content/drive/My Drive/imimic/wann/mnist_train_radiomics.npy" .

In [0]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import mnist
random.seed(1)
np.random.seed(1)
from __future__ import print_function
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K
from keras.callbacks import ModelCheckpoint


In [0]:
num_classes = 10

y_train = mnist.train_labels()
y_test = mnist.test_labels()
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


PREFIX = 'features'
x_train1 = np.load(f'mnist_train_{PREFIX}.npy')
x_test1 = np.load(f'mnist_test_{PREFIX}.npy')

PREFIX = 'radiomics'
x_train2 = np.load(f'mnist_train_{PREFIX}.npy')
x_test2 = np.load(f'mnist_test_{PREFIX}.npy')

x_train = np.concatenate((x_train1, x_train2), axis=1)
x_test = np.concatenate((x_test1, x_test2), axis=1)


In [0]:
model = Sequential()
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [0]:
batch_size = 128
epochs = 1000

filepath = 'weights-{epoch:02d}-{acc:.4f}-{val_acc:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# fits the model on batches with real-time data augmentation:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(x_test, y_test), callbacks=[checkpoint])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
!cp /content/weights-56-0.7169-0.7629.hdf5 "/content/drive/My Drive/imimic/wann/feature_weights.hdf5"